## 1.数据的存储
举例1：从TXT文档中加载数据，向量化后存储到Chroma数据库

In [9]:
from langchain_community.vectorstores import Chroma
from calendar import TextCalendar
from langchain_openai import OpenAIEmbeddings
import os
import dotenv
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 步骤1:创建一个TextLoader的实例，并将指定的文档加载
loader = TextLoader(
    file_path="./load/09-ai1.txt",
    encoding="utf-8",
)
docs = loader.load()
# 步骤2:创建文本拆分器
text_spliter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
split_docs = text_spliter.split_documents(docs)
print(type(split_docs))
# 步骤3:创建嵌入模型
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")
# 步骤4:将文档及嵌入模型传入到Chroma相关的结构中，进行数据的存储
db = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings_model,
    persist_directory="./load/chroma",
)
query = "人工智能的核心技术有哪些"
docs = db.similarity_search(query)
print(docs)

<class 'list'>
[Document(metadata={'source': './load/09-ai1.txt'}, page_content='3. 人工智能的核心技术\n3.1 机器学习\n机器学习是人工智能的核心技术之一，通过算法使计算机从数据中学习并做出决策。常见的机器学习算法包括监督学习、无监督学习和强化学习。监督学习通过标记数据进行训练，无监督学习则从未标记数据中寻找模式，强化学习则通过与环境交互来优化决策。\n3.2 深度学习\n深度学习是机器学习的一个子领域，通过多层神经网络进行特征提取和模式识别。深度学习在图像识别、自然语言处理、语音识别等领域取得了显著成果。常见的深度学习模型包括卷积神经网络（CNN）、循环神经网络（RNN）和长短期记忆网络（LSTM）。\n3.3 自然语言处理\n自然语言处理（NLP）是人工智能的一个重要分支，致力于使计算机能够理解和生成人类语言。NLP技术广泛应用于机器翻译、情感分析、文本分类等领域。近年来，基于深度学习的NLP模型（如BERT、GPT）在语言理解任务中取得了突破性进展。\n3.4 计算机视觉\n计算机视觉是人工智能的另一个重要分支，致力于使计算机能够理解和处理图像和视频。计算机视觉技术广泛应用于图像识别、目标检测、人脸识别等领域。深度学习模型（如CNN）在计算机视觉任务中取得了显著成果。\n\n4. 人工智能的应用领域\n4.1 医疗健康\n人工智能在医疗健康领域的应用包括疾病诊断、药物研发、个性化医疗等。通过分析医学影像和患者数据，人工智能可以帮助医生更准确地诊断疾病，提高治疗效果。\n4.2 金融\n人工智能在金融领域的应用包括风险评估、欺诈检测、算法交易等。通过分析市场数据和交易记录，人工智能可以帮助金融机构做出更明智的决策，提高运营效率。\n4.3 教育\n人工智能在教育领域的应用包括个性化学习、智能辅导、自动评分等。通过分析学生的学习数据，人工智能可以为学生提供个性化的学习建议，提高学习效果。\n4.4 交通\n人工智能在交通领域的应用包括自动驾驶、交通管理、智能导航等。通过分析交通数据和路况信息，人工智能可以帮助优化交通流量，提高交通安全。'), Document(metadata={'source': './load/09-ai1.txt'}, page_content='3. 

举例2

In [13]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
import dotenv
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
# 获取嵌入模型
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
# 加载文档并拆分（第1次拆分）
loader = CSVLoader("./load/03-load.csv", encoding='utf-8')
pages = loader.load_and_split()
#print(len(pages)) # 4
# 文本拆分（第2次拆分）
text_spliter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_spliter.split_documents(pages)
# 向量存储
db_path = './load/chroma_2'
db = Chroma.from_documents(docs, embeddings, persist_directory=db_path)


## 2.数据的检索

In [14]:
# 1.导入相关依赖
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
# 2.定义文档
raw_documents = [
Document(
    page_content="葡萄是一种常见的水果，属于葡萄科葡萄属植物。它的果实呈圆形或椭圆形，颜色有绿色、紫色、红色等多种。葡萄富含维生素C和抗氧化物质，可以直接食用或酿造成葡萄酒。",
    metadata={"source": "水果", "type": "植物"}
),
Document(
    page_content="白菜是十字花科蔬菜，原产于中国北方。它的叶片层层包裹形成紧密的球状，口感清脆微甜。白菜富含膳食纤维和维生素K，常用于制作泡菜、炒菜或煮汤。",
    metadata={"source": "蔬菜", "type": "植物"}
),
Document(
    page_content="狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。",
    metadata={"source": "动物", "type": "哺乳动物"}
),

Document(
    page_content="猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。",
    metadata={"source": "动物", "type": "哺乳动物"}
),
Document(
    page_content="人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。",
    metadata={"source": "生物", "type": "灵长类"}
),
Document(
    page_content="太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。",
    metadata={"source": "天文", "type": "恒星"}
),
Document(
    page_content="长城是中国古代的军事防御工程，总长度超过2万公里。它始建于春秋战国时期，秦朝连接各段，明朝大规模重修。长城是世界文化遗产和人类建筑奇迹。",
    metadata={"source": "历史", "type": "建筑"}
),
Document(
    page_content="量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。",
metadata={"source": "物理", "type": "科学"}
),
Document(
    page_content="《红楼梦》是中国古典文学四大名著之一，作者曹雪芹。小说以贾、史、王、薛四大家族的兴衰为背景，描绘了贾宝玉与林黛玉的爱情悲剧，反映了封建社会的种种矛盾。",
    metadata={"source": "文学", "type": "小说"}
),
Document(
    page_content="新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。",
    metadata={"source": "医学", "type": "病毒"}
)
]
# 3. 创建嵌入模型
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
# 4.创建向量数据库
db = Chroma.from_documents(
    documents=raw_documents,
    embedding=embedding,
    persist_directory="./asset/chroma-3",
)

1.相似性检索similarity


In [15]:
# 5. 检索示例（返回前3个最相关结果）
query = "哺乳动物"
docs = db.similarity_search(query, k=3) # k=3表示返回3个最相关文档
print(f"查询: '{query}' 的结果:")
for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

查询: '哺乳动物' 的结果:

结果 1:
内容: 猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 2:
内容: 狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据: {'source': '动物', 'type': '哺乳动物'}

结果 3:
内容: 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。
元数据: {'source': '生物', 'type': '灵长类'}


② 支持直接对问题向量查询（similarity_search_by_vector）

In [16]:
query = "哺乳动物"
embedding_vector = embedding.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector, k=3)
print(f"查询: '{query}' 的结果:")
for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

查询: '哺乳动物' 的结果:

结果 1:
内容: 猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 2:
内容: 狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 3:
内容: 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。
元数据: {'type': '灵长类', 'source': '生物'}


③ 相似性检索，支持过滤元数据（filter）

In [17]:
query = "哺乳动物"
docs = db.similarity_search(
query=query,
k=3,
filter={"source": "动物"})
for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")


结果 1:
内容: 猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据: {'source': '动物', 'type': '哺乳动物'}

结果 2:
内容: 狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据: {'type': '哺乳动物', 'source': '动物'}


④ 通过L2距离分数进行搜索（similarity_search_with_score）
说明：分数值越小，检索到的文档越和问题相似。分值取值范围：[0，正无穷]

In [18]:
docs = db.similarity_search_with_score(
"量子力学是什么?"
)
for doc, score in docs:
    print(f" [L2距离得分={score:.3f}] {doc.page_content} [{doc.metadata}]")

 [L2距离得分=0.182] 量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。 [{'type': '科学', 'source': '物理'}]
 [L2距离得分=0.447] 太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。 [{'type': '恒星', 'source': '天文'}]
 [L2距离得分=0.463] 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。 [{'source': '生物', 'type': '灵长类'}]
 [L2距离得分=0.488] 新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。 [{'source': '医学', 'type': '病毒'}]


⑤ 通过余弦相似度分数进行搜索（_similarity_search_with_relevance_scores）
说明：分数值越接近1（上限），检索到的文档越和问题相似。

In [19]:
docs = db._similarity_search_with_relevance_scores(
"量子力学是什么?"
)
for doc, score in docs:
    print(f"* [余弦相似度得分={score:.3f}] {doc.page_content} [{doc.metadata}]")

* [余弦相似度得分=0.871] 量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。 [{'source': '物理', 'type': '科学'}]
* [余弦相似度得分=0.684] 太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。 [{'type': '恒星', 'source': '天文'}]
* [余弦相似度得分=0.673] 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。 [{'type': '灵长类', 'source': '生物'}]
* [余弦相似度得分=0.655] 新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。 [{'type': '病毒', 'source': '医学'}]


⑥ MMR（最大边际相关性，max_marginal_relevance_search）
MMR 是一种平衡 相关性 和 多样性 的检索策略，避免返回高度相似的冗余结果。

In [20]:
docs = db.max_marginal_relevance_search(
query="量子力学是什么",
lambda_mult=0.8, # 侧重相似性
)
print("🔍 关于【量子力学是什么】的搜索结果：")
print("=" * 50)
for i, doc in enumerate(docs):
    print(f"\n📖 结果 {i+1}:")
    print(f"📌 内容: {doc.page_content}")
    print(f"🏷 标签: {', '.join(f'{k}={v}' for k, v in doc.metadata.items())}")

🔍 关于【量子力学是什么】的搜索结果：

📖 结果 1:
📌 内容: 量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。
🏷 标签: source=物理, type=科学

📖 结果 2:
📌 内容: 太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。
🏷 标签: type=恒星, source=天文

📖 结果 3:
📌 内容: 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。
🏷 标签: source=生物, type=灵长类

📖 结果 4:
📌 内容: 新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。
🏷 标签: type=病毒, source=医学


## 6、检索器(召回器) Retrievers

In [ ]:
# 1.导入相关依赖
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import os
import dotenv
dotenv.load_dotenv()
# 2.定义文档加载器
loader = TextLoader(file_path='./asset/load/09-ai1.txt',encoding="utf-8")
# 3.加载文档
documents = loader.load()
# 4.定义文本切割器
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# 5.切割文档
docs = text_splitter.split_documents(documents)
# 6.定义嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
embeddings = OpenAIEmbeddings(
model="text-embedding-3-large"
)

# 7.将文档存储到向量数据库中
db = FAISS.from_documents(docs, embeddings)
# 8.从向量数据库中得到检索器
retriever = db.as_retriever()
# 9.使用检索器检索
docs = retriever.invoke("深度学习是什么？")
print(len(docs))
# 10.得到结果
for doc in docs:
    print(f"⭐{doc}")